In [ ]:
from ensemble_generator.perturber_func import perturb_interface, perturb_orient_vMF, perturb_fault_interface_vMF

In [ ]:
#############################
#      Names and labels     #
#############################

model_label = 'Geomodel_demo'
source_geomodeller = True  # True or False
#############################
#           Paths           #
#############################

source_geomodeller=False
path_to_geomodeller = 'C:/GeoModeller/GeoModeller4.0.7_x64_27eee3dc31ba'

path_to_model = '../test-data/'  # where the output from map2loop lives
path_to_scratch='../egen_runs/'  # where we will store the perturbed models

#############################
#           Files           #
#############################
DEM=False
DTM_name = 'dtm_rp.tif'  # , just the filename, not the path, includes file extension
file_type='contacts'
#############################
#       Ensemble par        #
#############################

egen_runs = 1
save_faults = True  # True/False: do we want faults included?

#############################
#       Voxet parameters    #
#############################

litho = True  # True or comment out
scalar = False  # True or comment out
scalar_grads = False  # True or comment out

# Voxet parameters
nx = 50
ny = 50
nz = 50

#############################
#      Perturbing params    #
#############################

# interface
error_gps = 10
distribution = 'uniform'
DEM = False  # True or none
# orientations
kappa = 50
error_gps = 10
loc_distribution = 'uniform'

#############################
#       Compute params      #
#############################

series_list = 'all'  # list of series to calculate
fault_list = 'all'  # (['Fault_12644', 'Fault_2235', 'Fault_11442', 'Fault_3496', 'Fault_5298', 'Fault_12647']) # list of faults to calculate or 'all'
krig_range = None  # kriging range - set to None for default values
interface = None  # nugget effect on interface; larger values = smoother model (less adherence to the data) default = 0.000001. Set to None for default value
orientation = None  # nugget effect on orientation data; larger values = smoother model (less adherence to the data) default = 0.01. Set to None for default value
drift = None  #

#############################
#       Summary stats       #
#############################

card = True  # True or False - export a voxet representing model cardinality
ent = True  # True or False - export a voxet representing model entropy
propor = True  # True or False - export a voxet representing the proportions of modelled lithologies
export = True  # Export voxet - Can be 'False' if the arrays produced by the summary stats function are being passed directly to a downstream process
air = True  # is there an 'air' layer in the model? Will be yes if there is a DTM and/or the surface topo is not at the top of the model volume. Important for geodiversity and proportion calculations.


In [ ]:
egen_runs=5
path_to_scratch

In [ ]:
import os
import shutil
import pandas as pd
import numpy as np

if(not os.path.isdir(path_to_scratch)):
    os.mkdir(path_to_scratch)
if(not os.path.isdir(path_to_scratch+'/../gocad')):
    os.mkdir(path_to_scratch+'/../gocad')

for s in range(egen_runs):
    path_to_scratch2=path_to_scratch+'model_{0:06d}'.format(s)
    if(not os.path.isdir(path_to_model+path_to_scratch2)):
        os.mkdir(path_to_model+path_to_scratch2)
    if(not os.path.isdir(path_to_model+path_to_scratch2+'/output')):
        os.mkdir(path_to_model+path_to_scratch2+'/output')
    if(not os.path.isdir(path_to_model+path_to_scratch2+'/tmp')):
        os.mkdir(path_to_model+path_to_scratch2+'/tmp')
    if(not os.path.isdir(path_to_model+path_to_scratch2+'/dtm')):
        os.mkdir(path_to_model+path_to_scratch2+'/dtm')
    if(not os.path.isdir(path_to_model+path_to_scratch2+'/vtk')):
        os.mkdir(path_to_model+path_to_scratch2+'/vtk')

files=[
    'tmp/all_sorts_clean.csv','tmp/all_sorts_clean.csv', # (stratigraphy)
    'tmp/raw_contacts.csv','tmp/raw_contacts.csv', # (undecimated contacts)
    'output/contacts_{}.csv', 'output/contacts_clean.csv', #(all contacts)
    'output/fault_displacements3.csv','output/fault_displacements3.csv', #(local estimates of fault displacements)
    'output/faults.csv','output/faults.csv', #(fault trace info)
    'output/fault_orientations_{}.csv','output/fault_orientations.csv', #(table of which faults truncate which other faults)
    'output/fault-fault-relationships.csv','output/fault-fault-relationships.csv', #(table of which faults truncate which strat units)
    'output/group-fault-relationships.csv','output/group-fault-relationships.csv', #(table of which faults truncate which strat units)
    'output/fault_dimensions.csv','output/fault_dimensions.csv', #(fault extent ellipsoid data)
    'output/fault_dimensions.csv','output/fault_dimensions.csv', #(fault orientationdata)
    'tmp/super_groups.csv','tmp/super_groups.csv', #(which sets groups should be co-interpolated)
    'tmp/bbox.csv','tmp/bbox.csv', # (bounding box in whatever metre-based projection system data is in)
    'output/formation_summary_thicknesses.csv','output/formation_summary_thicknesses.csv', #(estimated summary formation thickness info)
    'dtm/dtm_rp.tif','dtm/dtm_rp.tif', #(dtm as geotif, larger dimensions than bbox to make sure reprojected raster covers the whole area)
    'tmp/bbox.csv','tmp/bbox.csv' # bounding box
]

In [ ]:
perturb_interface(path_to_model+'output',egen_runs, error_gps, file_type, loc_distribution, DEM, source_geomodeller)
perturb_orient_vMF(path_to_model+'output',egen_runs, kappa, error_gps,file_type, loc_distribution, DEM, source_geomodeller)
perturb_fault_interface_vMF(path_to_model+'output',egen_runs, kappa)

In [ ]:
pluton_form='domes'
for s in range(egen_runs):
    path_to_scratch2=path_to_scratch+'model_{0:06d}'.format(s)+'/'
    all_orientations = pd.read_csv(path_to_model+'output/orientations_{}.csv'.format(s),",")

    if( os.path.exists(path_to_model+'output/ign_orientations_{}'+pluton_form.format(s)+'.csv')):
        intrusive_orientations = pd.read_csv(path_to_model+'output/ign_orientations_'+pluton_form+'.csv',",")
        all_orientations = pd.concat([all_orientations,intrusive_orientations],sort = False)
    else:
        print('No intrusive orientations available for merging.')
    
    all_orientations.to_csv(path_to_scratch2+'/output/orientations_clean.csv')
    for a in range (0,int(len(files)),2):
        #print(path_to_model+files[a],path_to_scratch+files[a+1])
        shutil.copy2(path_to_model+files[a].format(s),path_to_scratch2+files[a+1])

    #shutil.make_archive(path_to_scratch+'/../scratch', 'zip', path_to_scratch)


In [ ]:
def write_vol_gocad(model, file_path,file_name, data_label, nsteps, real_coords=True):
    """
    Writes out the model as a 3d volume grid in GOCAD VOXET object format

    Parameters
    ----------
    model : GeologicalModel object
        Geological model to export
    file_name : string
        Name of file that model is exported to, including path, but without the file extension
    data_label : string
        A data label to insert into export file
    nsteps : np.array([num-x-steps, num-y-steps, num-z-steps])
        3d array dimensions

    Returns
    -------
    True if successful

    """
    # Define grid spacing in model scale coords
    loop_X = np.linspace(model.bounding_box[0, 0], model.bounding_box[1, 0], nsteps[0])
    loop_Y = np.linspace(model.bounding_box[0, 1], model.bounding_box[1, 1], nsteps[1])
    loop_Z = np.linspace(model.bounding_box[0, 2], model.bounding_box[1, 2], nsteps[2])

    # Generate model values in 3d grid
    xx, yy, zz = np.meshgrid(loop_X, loop_Y, loop_Z, indexing='ij')
    # xyz is N x 3 vector array
    xyz = np.array([xx.flatten(), yy.flatten(), zz.flatten()]).T
    vals = model.evaluate_model(xyz, scale=False)
    # Use FORTRAN style indexing for GOCAD VOXET
    vol_vals = np.reshape(vals, nsteps, order='F')
    bbox = model.bounding_box[:]
        
    # Convert bounding box to real world scale coords
    if real_coords:
        model.rescale(np.int64)
    print(type(vals[0]))
    # If integer values
    if (type(vals[0]) is np.int64 or type(vals[0]) is np.int32 ):
        d_type = np.int8
        no_data_val = None
        prop_esize = 1
        prop_storage_type = "Octet"

    # If float values
    elif type(vals[0]) is np.float32:
        d_type = np.dtype('>f4')
        no_data_val = -999999.0
        prop_esize = 4
        prop_storage_type = "Float"
    else:
        print("Cannot export volume to GOCAD VOXET file: Unsupported type {}".format(type(vals[0])))
        return False

    # Write out VOXET file
    vo_filename = file_name + ".vo"
    data_filename = file_name + "@@"
    try:
        with open(file_path+vo_filename, "w") as fp:
            fp.write("""GOCAD Voxet 1
HEADER {{
name: {name}
}}
AXIS_O 0.000000 0.000000 0.000000
AXIS_U 1.000000 0.000000 0.000000
AXIS_V 0.000000 1.000000 0.000000
AXIS_W 0.000000 0.000000 1.000000
AXIS_MIN {axismin1} {axismin2} {axismin3}
AXIS_MAX {axismax1} {axismax2} {axismax3}
AXIS_N {nsteps1} {nsteps2} {nsteps3}
AXIS_D {xdim} {ydim} {zdim}
AXIS_NAME "X" "Y" "Z"
AXIS_UNIT "m" "m" "m"
AXIS_TYPE even even even
PROPERTY 1 {propname}
PROPERTY_CLASS 1 {propname}
PROP_UNIT 1 {propname}
PROPERTY_CLASS_HEADER 1 {propname} {{
}}
PROPERTY_SUBCLASS 1 QUANTITY {prop_storage_type}
""".format(name=os.path.basename(file_name),
                nsteps1=nsteps[0], nsteps2=nsteps[1], nsteps3=nsteps[2],
                axismin1=bbox[0, 0], axismin2=bbox[0, 1], axismin3=bbox[0, 2],
                axismax1=bbox[1, 0], axismax2=bbox[1, 1], axismax3=bbox[1, 2],
                propname=data_label, prop_storage_type=prop_storage_type,
                xdim=(bbox[0, 0]-bbox[1, 0])/nsteps[0],
                ydim=(bbox[0, 1]-bbox[1, 1])/nsteps[1],
                zdim=(bbox[0, 2]-bbox[1, 2])/nsteps[2]))
            if no_data_val is not None:
                fp.write("PROP_NO_DATA_VALUE 1 {no_data_val}\n".format(no_data_val=no_data_val))
            fp.write("""PROP_ETYPE 1 IEEE
PROP_FORMAT 1 RAW
PROP_ESIZE 1 {prop_esize}
PROP_OFFSET 1 0
PROP_FILE 1 {prop_file}
END\n""".format(prop_file=data_filename, prop_esize=prop_esize))
    except IOError as exc:
        print("Cannot export volume to GOCAD VOXET file {}: {}".format(vo_filename, str(exc)))
        return False

    # Write out accompanying binary data file
    export_vals = np.array(vol_vals, dtype=d_type)
    try:
        with open(file_path+data_filename, "wb") as fp:
            export_vals.tofile(fp)
    except IOError as exc:
        print("Cannot export volume to GOCAD VOXET data file {}: {}".format(data_filename, str(exc)))
        return False
    return True


In [ ]:
for s in range(egen_runs):
    path_to_scratch2=path_to_scratch+'model_{0:06d}'.format(s)
    tmp_path=path_to_scratch2+'/tmp/'
    vtk_path=path_to_scratch2+'/vtk/'
    output_path=path_to_scratch2+'/output/'
    dtm_path=path_to_scratch2+'/dtm/'
    fault_file=path_to_scratch2+'/output/faults.csv'
    test_data_path=path_to_scratch2
    gocad_path=path_to_scratch+'/../gocad/'
    print(output_path)
    if(True):
        #model_base=-8200

        import random
        from datetime import datetime
        from LoopStructural import GeologicalModel
        import lavavu
        from LoopStructural.visualisation import LavaVuModelViewer
        from LoopStructural import GeologicalModel
        import logging
        logging.getLogger().setLevel(logging.ERROR)

        nowtime=datetime.now().isoformat(timespec='minutes')   
        model_name='leaflet'+'_'+nowtime.replace(":","-").replace("T","-")
        os.mkdir(vtk_path+model_name)
        filename=vtk_path+model_name+'/'+'win_'+'surface_name_{}.vtk'


        fault_params = {'interpolatortype':'FDI',
                        'nelements':3e4,
                        #'data_region':.3,
                        'solver':'pyamg',
        #                 overprints:overprints,
                        'cpw':10,
                        'npw':10}
        foliation_params = {'interpolatortype':'FDI' , # 'interpolatortype':'PLI',
                            'nelements':0.2e5,  # how many tetras/voxels
                            'buffer':0.8,  # how much to extend nterpolation around box
                            'solver':'pyamg',
                            'damp':True}

        if(not os.path.exists(fault_file)):
            f=open(output_path + '/fault_displacements3.csv','w')
            f.write('X,Y,fname,apparent_displacement,vertical_displacement,downthrow_dir\n')
            f.close()
            f=open(output_path + '/fault_orientations.csv','w')
            f.write('X,Y,Z,DipDirection,dip,DipPolarity,formation\n')
            f.close()
            f=open(output_path + '/faults.csv','w')
            f.write('X,Y,Z,formation\n')
            f.close()
            f=open(output_path + '/fault-fault-relationships.csv','w')
            f.write('fault_id\n')
            f.close()
            f=open(output_path + '/group-fault-relationships.csv','w')
            f.write('group\n')
            f.close()

            model, m2l_data = GeologicalModel.from_map2loop_directory(test_data_path,
                                                                  skip_faults=True,
                                                                  fault_params=fault_params,
                                                                  foliation_params=foliation_params)
        else:
            model, m2l_data = GeologicalModel.from_map2loop_directory(test_data_path,
                                                                  skip_faults=False,
                                                                  fault_params=fault_params,
                                                                  foliation_params=foliation_params,
                                                                  unconformities=True)

        """view = LavaVuModelViewer(model,vertical_exaggeration=2) 
        #view.set_zscale(2)
        view.nsteps=np.array([50,50,50])
        view.add_model_surfaces(filename=filename)
        for sg in model.feature_name_index:
            if( 'super' in sg):
                view.add_data(model.features[model.feature_name_index[sg]])
        #view.lv.webgl(vtk_path+model_name)
        view.nsteps = np.array([200,200,200])
        view.add_model()
        view.interactive()  """
        write_vol_gocad(model, gocad_path,'GOCAD_LITHO'+'_{0:06d}'.format(s), 'lithology', [nx,ny,nz], real_coords=False)



In [ ]:
from ensemble_generator.egen_summary_stats import stats_gocad_voxet

litho_df, card_data, ent_data, propor_data=stats_gocad_voxet(gocad_path, 'GOCAD_LITHO', model_label='lithology', card=True, ent=True, propor=True, export=True, air=False)
